In [4]:
import pytesseract
import cv2
import pyocr 
from PIL import Image
from PIL.ImageOps import invert
import matplotlib.pyplot as plt
import time
import numpy as np
import pytesseract

import imutils
from imutils.perspective import four_point_transform

In [5]:
cv2.__version__

'3.4.2'

In [6]:
img = cv2.imread("license/AA.jpg", cv2.IMREAD_COLOR)
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
# 1440*900
bili = 800/img_gray.shape[0]
re_img_gray = cv2.resize(img_gray, dsize=(0,0), fx=bili, fy = bili)

_, img_gray_th = cv2.threshold(re_img_gray, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)
poin_img = cv2.imread("license/A_point.jpg", cv2.IMREAD_GRAYSCALE)
_, poin_img_th = cv2.threshold(poin_img, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)

In [8]:
w, h = poin_img.shape[::-1]
res = cv2.matchTemplate(img_gray_th, poin_img_th, cv2.TM_CCORR_NORMED) 
threshold = res.max()
loc = np.where(res >= threshold)
for x,y in zip(*loc[::-1]):
    cv2.rectangle(re_img_gray, (x,y), (x+w, y+h), (0,0,255), 1)

ValueError: too many values to unpack (expected 2)

In [9]:
loc

NameError: name 'loc' is not defined

In [10]:
cv2.imshow("poin_img_th",re_img_gray)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [11]:
image = cv2.imread('license/AA.jpg')
width1, height1 = image.shape[1],image.shape[0]

# bili = 800/image.shape[0]
# re_img_gray = cv2.resize(image, dsize=(0,0), fx=bili, fy = bili)

gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
#高斯滤波
blurred = cv2.GaussianBlur(gray, (5, 5), 0)
#自适应二值化方法
blurred=cv2.adaptiveThreshold(blurred,255,cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY,51,2)
'''
adaptiveThreshold函数：第一个参数src指原图像，原图像应该是灰度图。
    第二个参数x指当像素值高于（有时是小于）阈值时应该被赋予的新的像素值
    第三个参数adaptive_method 指： CV_ADAPTIVE_THRESH_MEAN_C 或 CV_ADAPTIVE_THRESH_GAUSSIAN_C
    第四个参数threshold_type  指取阈值类型：必须是下者之一 CV_THRESH_BINARY,• CV_THRESH_BINARY_INV
    第五个参数 block_size 指用来计算阈值的象素邻域大小: 3, 5, 7, ...
    第六个参数param1指与方法有关的参数。对方法CV_ADAPTIVE_THRESH_MEAN_C 和 CV_ADAPTIVE_THRESH_GAUSSIAN_C， 
    它是一个从均值或加权均值提取的常数, 尽管它可以是负数。
'''
#这一步可有可无，主要是增加一圈白框，以免刚好卷子边框压线后期边缘检测无果。好的样本图就不用考虑这种问题
blurred=cv2.copyMakeBorder(blurred,5,5,5,5,cv2.BORDER_CONSTANT,value=(255,255,255))

edged = cv2.Canny(blurred, 10, 100)
cnts = cv2.findContours(edged, cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
cnts = cnts[0] if imutils.is_cv2() else cnts[1]
docCnt = None

if len(cnts) > 0:
    # 将轮廓按大小降序排序
    cnts = sorted(cnts, key=cv2.contourArea, reverse=True)
    # 对排序后的轮廓循环处理
    for c in cnts:
        # 获取近似的轮廓
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.02 * peri, True)
        # 如果近似轮廓有四个顶点，那么就认为找到了答题卡
        if len(approx) == 4:
            docCnt = approx
            break
            
newimage = re_img_gray.copy()
for i in docCnt:
    #circle函数为在图像上作图，新建了一个图像用来演示四角选取
    cv2.circle(newimage, (i[0][0],i[0][1]), 10, (255, 0, 0), -1)
    
paper = four_point_transform(image, docCnt.reshape(4, 2))
warped = four_point_transform(gray, docCnt.reshape(4, 2))

# 对灰度图应用二值化算法
thresh=cv2.adaptiveThreshold(warped,255,cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY,53,2)
#重塑可能用到的图像
thresh = cv2.resize(thresh, (width1, height1), cv2.INTER_LANCZOS4)
paper = cv2.resize(paper, (width1, height1), cv2.INTER_LANCZOS4)
warped = cv2.resize(warped, (width1, height1), cv2.INTER_LANCZOS4)
#均值滤波
# ChQImg = cv2.blur(thresh, (23, 23))
#二进制二值化
# ChQImg = cv2.threshold(ChQImg, 100, 225, cv2.THRESH_BINARY)[1]

cv2.imshow("edged",edged)
cv2.imshow("gray",gray)
cv2.imshow("newimage",newimage)
cv2.imshow("thresh",thresh)
# cv2.imshow("paper",paper)
cv2.imshow("warped",warped)
cv2.imwrite("license/warpedAA.jpg",warped)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [202]:
re_img_gray.shape

(800, 525, 3)

In [50]:
# -*- coding: utf-8 -*-
import pyzbar
from pyzbar.pyzbar import decode
from pyzbar.pyzbar import ZBarSymbol
import encodings

code2D = cv2.imread('license/QRcode.png')
result = decode(code2D, symbols=[ZBarSymbol.QRCODE])
for info in result:
    print(info.type, "**",info.data.decode("utf-8"))

QRCODE ** 统一社会信用代码：91510100667583777F
企业注册号：510109000304280
名称：成都西辰软件有限公司
登记机关：成都市高新工商行政管理局
登记日期：2017-02-10
企业信用信息公示系统网址：http://gsxt.scaic.gov.cn  http://gsxt.cdcredit.gov.cn


In [46]:
information = info.data.decode('utf-8')
information

'统一社会信用代码：9144011476769188XG|注册号：440121000022135|数据查询链接：http://www.gzonline.gov.cn/cri/BusinessMobile.aspx?Id=9031F652A9AAA814D6C2E1CE7450E03E'

In [44]:
result

[]

ModuleNotFoundError: No module named 'png'

In [27]:
dir(pyqrcode.create)

['__annotations__',
 '__call__',
 '__class__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__globals__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__kwdefaults__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [58]:
ss = "喜欢"
ss.encode("utf-8").decode("utf-8")

'喜欢'